In [46]:
import numpy as np
import pandas as pd
import sklearn as sk
# from sklearn.trees import RandomForestClassifier
from sklearn.svm import SVC

In [47]:

df = pd.read_csv("traffic.csv",parse_dates=True, index_col='DateTime')


# extract year from date
df['Year'] = pd.Series(df.index).apply(lambda x: x.year).to_list()

# extract month from date
df['Month'] = pd.Series(df.index).apply(lambda x: x.month).to_list()

# extract day from date
df['Day'] = pd.Series(df.index).apply(lambda x: x.day).to_list()

# extract hour from date
df['Hour'] = pd.Series(df.index).apply(lambda x: x.hour).to_list()

grouped = df.groupby(df.Junction)
df1 = grouped.get_group(1)
df2 = grouped.get_group(2)
df3 = grouped.get_group(3)
df4 = grouped.get_group(4)
# print(df1)






In [84]:
ratio = 80
dataDivide = len(df1) * ratio // 100

x1 = df1.iloc[:, 3:].values.tolist()
x2 = sk.utils.validation.column_or_1d(df1.iloc[:, 1:2].values.tolist())
# print(x1)

# from sklearn import preprocessing
# import numpy as np
# x2 = preprocessing.normalize([x2])[0]
# print(min(x2))
from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit([x2])
x2 = transformer.transform([x2])[0]
print(min(x2))
print(max(x2))

x1Train, x1Test, x2Train, x2Test = x1[:dataDivide], x1[
    dataDivide:], x2[:dataDivide], x2[dataDivide:]

0.0008182159646018199
0.02552833809557678


In [79]:
# print(x1Train,x2Train)

In [80]:
# model = SVC(random_state=1)
# model.fit(x1Train, x2Train)
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(x1Train, x2Train)

ValueError: Found input variables with inconsistent numbers of samples: [11673, 1]

In [52]:
print(rf.score(x1Test, x2Test))

0.2315542620933242
